# **Hypothesis Testing**  

This notebook performs hypothesis testing for evaluating whether the introduction of VAR significantly affected home advantage across six different football leagues between the 2014–15 and 2024–25 seasons.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, pearsonr

df = pd.read_csv("data/main_merged_and_cleaned.csv")
df.head()

,id,team,country,league,season_start,season_end,final_league_pos,home_matches,home_wins,home_draws,...,home_yellow_cards_won,home_yellow_cards_conceded,home_red_cards_won,home_red_conceded,away_fouls_won_avg,away_fouls_conceded_avg,away_yellow_cards_won,away_yellow_cards_conceded,away_red_cards_won,away_red_cards_conceded
0,1,Hoffenheim,de,Bundesliga,2014,2015,8,17,9,3,...,25.294118,20.000000,2.352941,0.000000,181.176471,190.588235,19.411765,18.823529,1.176471,1.176471
1,2,Udinese,it,Serie A,2014,2015,16,19,6,5,...,25.789474,21.052632,0.526316,1.578947,136.842105,150.000000,20.526316,26.842105,2.105263,1.578947
2,3,Bayern Munich,de,Bundesliga,2014,2015,1,17,14,1,...,13.529412,6.470588,0.588235,1.176471,162.941177,140.000000,16.470588,15.294118,0.588235,0.000000
3,4,Augsburg,de,Bundesliga,2014,2015,5,17,9,4,...,24.705882,15.294118,0.588235,1.176471,150.000000,161.176471,18.235294,20.588235,1.764706,0.588235
4,5,Hellas Verona,it,Serie A,2014,2015,13,19,7,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Hypothesis Definition**

We aim to answer whether the introduction of Video Assistant Referee changed home advantage in football.

* Null Hypothesis (H₀): The implementation of VAR is not significantly affecting home advantage.

* Alternative Hypothesis (H₁): The implementation of VAR is significantly reducing home advantage.

We will test this using independent samples t-tests, correlation differences between home advantage and final league ranking and effect size measures comparing pre-VAR and post-VAR.




**Prepare Variables for Testing**

We use the same VAR_era construction as in the Exploratory Data Analysis.

We assign:
- 0 → Pre-VAR  
- 1 → Post-VAR  


In [2]:
var_start = {
    "Bundesliga": 2017,
    "Serie A": 2017,
    "La Liga": 2018,
    "Premier League": 2019,
    "Ligue 1": 2018,
    "Süper Lig": 2018
}

df['VAR_era'] = df.apply(
    lambda x: 1 if x['season_start'] >= var_start.get(x['league'], 9999) else 0,
    axis=1
)

df['VAR_era'].value_counts()

,count
VAR_era,
1,835
0,446


**Test 1: Difference in Home Wins (Pre-VAR vs Post-VAR)**

We test whether average home wins per season significantly changed after VAR introduction.

Test (Independent Samples t-test):  
H₀: μ_pre = μ_post  
H₁: μ_pre > μ_post


In [3]:
def welch_df(a, b):
    var_a = np.var(a, ddof=1)
    var_b = np.var(b, ddof=1)
    n_a = len(a)
    n_b = len(b)

    numerator = (var_a/n_a + var_b/n_b)**2
    denominator = ((var_a**2)/((n_a**2)*(n_a-1))) + ((var_b**2)/((n_b**2)*(n_b-1)))

    return numerator / denominator

In [4]:
pre = df[df['VAR_era'] == 0]['home_wins']
post = df[df['VAR_era'] == 1]['home_wins']

t_stat, p_value = ttest_ind(pre, post, equal_var=False)

dfree = welch_df(pre, post)

print("t-statistic:", t_stat)
print("p-value:", p_value)
print("Degrees of Freedom:", dfree)
print("Mean Pre-VAR:", pre.mean())
print("Mean Post-VAR:", post.mean())

t-statistic: 2.6991274779550083
p-value: 0.007081308470044509
Degrees of Freedom: 908.0632709206855
Mean Pre-VAR: 8.502242152466367
Mean Post-VAR: 7.9664670658682635


*The mean number of home wins decreases after VAR. The t-test shows a statistically significant difference (p < 0.01), so we reject H₀. The decline is small in magnitude, indicating a limited but measurable reduction in home advantage.*

**Test 2: Home Win Rate (Pre vs Post VAR)**

We repeat the test using *home_win_rate*, a normalized measure reflecting performance quality.

Test (Independent Samples t-test):  
H₀: μ_pre = μ_post  
H₁: μ_pre > μ_post

In [5]:
pre = df[df['VAR_era'] == 0]['home_win_rate']
post = df[df['VAR_era'] == 1]['home_win_rate']

t_stat, p_value = ttest_ind(pre, post, equal_var=False)

dfree = welch_df(pre, post)

print("t-statistic:", t_stat)
print("p-value:", p_value)
print("Degrees of Freedom:", dfree)
print("Mean Pre-VAR:", pre.mean())
print("Mean Post-VAR:", post.mean())

t-statistic: 2.483938895738365
p-value: 0.013172697413022806
Degrees of Freedom: 910.5135704938583
Mean Pre-VAR: 0.4613280762556053
Mean Post-VAR: 0.4347601242742515


*Home win rate also shows a meaningful decrease (p < 0.02). Although the effect size is small, the result aligns with Test 1 and supports that home teams win slightly less often after VAR.*

**Test 3: Home Points per Match (Pre vs Post VAR)**

This metric incorporates wins + draws and is one of the strongest indicators of team performance at home.

Test (Independent Samples t-test):  
H₀: μ_pre = μ_post  
H₁: μ_pre > μ_post

In [6]:
pre = df[df['VAR_era'] == 0]['home_points_per_match']
post = df[df['VAR_era'] == 1]['home_points_per_match']

t_stat, p_value = ttest_ind(pre, post, equal_var=False, nan_policy='omit')

dfree = welch_df(pre, post)

print("t-statistic:", t_stat)
print("p-value:", p_value)
print("Degrees of Freedom:", dfree)
print("Mean Pre-VAR:", pre.mean())
print("Mean Post-VAR:", post.mean())

t-statistic: 2.3379271503411707
p-value: 0.019606665940797983
Degrees of Freedom: 913.5705880204039
Mean Pre-VAR: 1.6275505617977528
Mean Post-VAR: 1.5586467065868261


*Home points per match decline modestly and the difference is statistically meaningful (p < 0.02). This suggests that the reduction in home advantage is consistent across multiple performance metrics.*

**Test 4: Home Advantage Index**

This composite index was constructed in the Exploratory Data Analysis and reflects normalized home–away differences across wins, goals, xG, fouls, cards, and penalties.

Test (Independent Samples t-test):  
H₀: μ_pre = μ_post  
H₁: μ_pre > μ_post  




In [7]:
cols_home_away = {
    'home_wins': 'away_wins',
    'home_points_per_match': 'away_points_per_match',
    'home_goals_for': 'away_goals_for',
    'home_goals_against': 'away_goals_against',
    'home_xg_avg': 'away_xg_avg',
    'home_fouls_won_avg': 'away_fouls_won_avg',
    'home_yellow_cards_won': 'away_yellow_cards_won',
    'home_red_cards_won': 'away_red_cards_won',
    'home_penalties_won': 'away_penalties_won',
    'home_penalties_conceded': 'away_penalties_conceded'
}

for h, a in cols_home_away.items():
    df[f"diff_{h}"] = df[h] - df[a]

diff_cols = [col for col in df.columns if col.startswith("diff_")]

df[diff_cols] = (df[diff_cols] - df[diff_cols].mean()) / df[diff_cols].std()

df['home_adv_index'] = df[diff_cols].mean(axis=1)

df[['team', 'season_start', 'home_adv_index']].head()


,team,season_start,home_adv_index
0,Hoffenheim,2014,0.561405
1,Udinese,2014,0.053619
2,Bayern Munich,2014,-0.314163
3,Augsburg,2014,0.631492
4,Hellas Verona,2014,-0.091441


In [8]:
pre = df[df['VAR_era'] == 0]['home_adv_index']
post = df[df['VAR_era'] == 1]['home_adv_index']

t_stat, p_value = ttest_ind(pre, post, equal_var=False, nan_policy='omit')

dfree = welch_df(pre, post)

print("t-statistic:", t_stat)
print("p-value:", p_value)
print("Degrees of Freedom:", dfree)
print("Mean Pre-VAR:", pre.mean())
print("Mean Post-VAR:", post.mean())

t-statistic: 4.384608636806646
p-value: 1.2950039931928876e-05
Degrees of Freedom: 926.7664088774051
Mean Pre-VAR: 0.08311869460801288
Mean Post-VAR: -0.03314799916267114


*The Home Advantage Index drops from positive (pre-VAR) to negative (post-VAR), indicating a shift toward more balanced home–away conditions. The difference is meaningful (p < 0.001), providing evidence that VAR is associated with a reduction in overall home advantage.*

**Test 5: Correlation Change — Home Advantage Index vs League Ranking**

We test whether the relationship between home advantage and final ranking changed after VAR.

Test:\
Pearson Correlation (Pre-VAR)  
Pearson Correlation (Post-VAR)

Then compare:

- Did correlation magnitude change?
- Did correlation sign change?

In [9]:
pre_df = df[df['VAR_era'] == 0]
post_df = df[df['VAR_era'] == 1]

r_pre, p_pre = pearsonr(pre_df['home_adv_index'], pre_df['final_league_pos'])
r_post, p_post = pearsonr(post_df['home_adv_index'], post_df['final_league_pos'])

print("Pre-VAR Pearson r:", r_pre)
print("Pre-VAR p-value:", p_pre)
print()
print("Post-VAR Pearson r:", r_post)
print("Post-VAR p-value:", p_post)

Pre-VAR Pearson r: -0.1048727861902569
Pre-VAR p-value: 0.02678251567855079

Post-VAR Pearson r: -0.10608734841534337
Post-VAR p-value: 0.002143364775600884


*The correlation between home advantage and league ranking remains weak and negative in both eras, with almost identical magnitudes. This indicates that VAR did not materially change how home advantage relates to final league performance.*

**Overall Summary**\
*Across all major performance indicators-wins, win rate, points per match, and the composite home advantage index-home advantage declines after VAR. The effects are statistically significant but small in size. The relationship between home advantage and final league ranking remains essentially unchanged. Overall, results suggest that VAR contributes to a modest but consistent reduction in home advantage.*